# Introduction

This is my Notebook on ESO competition to predict the Battery ouput  I will do the followoing:
* Perform **[EDA and Feature Engineering](#eda)** to clean the dataset, add new features to try to improve the accuracy of the prediction and understand  seasonality for Baterry Forecast
* Use **[XGBoost](#xgboost)** model to predict Battery output.

* Use **[Prophet](#prophet)** model to predict Battery output.
* Use **[LightGBM](#lgbm)**   to predict Battery output.
* Compare the results from the different models, draw key insights and provide final output for submission.

#  Table of contents
* [EDA and Feature Engineering](#eda_fe)
* [XGBoost](#xgboost)
* [Prophet](#prophet)
* [LightGBM](#lgbm)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("train_data.csv", index_col= 0)

In [3]:
df

,UTC_Settlement_DateTime,battery_output,East_Midlands_price,Eastern_England_price,London_price,Merseyside_and_Northern_Wales_price,North_Western_England_price,Northern_Scotland_price,South_Western_England_price,Southern_England_price,...,is_daySouthhampton - Portsmouth_weather,locationSouthhampton - Portsmouth_weather,populationSouthhampton - Portsmouth_weather,latitudeSouthhampton - Portsmouth_weather,longitudeSouthhampton - Portsmouth_weather,generationtime_msSouthhampton - Portsmouth_weather,utc_offset_secondsSouthhampton - Portsmouth_weather,timezoneSouthhampton - Portsmouth_weather,timezone_abbreviationSouthhampton - Portsmouth_weather,elevationSouthhampton - Portsmouth_weather
id,,,,,,,,,,,,,,,,,,,,,
0,8/10/2020 23:30,1.528,5.985,6.279,5.985,6.5835,6.279,7.1820,6.8880,6.279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8/11/2020 0:00,0.178,6.300,6.615,6.300,6.9300,6.615,7.5600,7.2450,6.615,...,0.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2,8/11/2020 0:30,-1.242,6.300,6.615,6.300,6.9300,6.615,7.5600,7.2450,6.615,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8/11/2020 1:00,2.104,5.985,6.279,5.985,6.5835,6.279,7.1820,6.8880,6.279,...,0.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
4,8/11/2020 1:30,-0.664,5.985,6.279,5.985,6.5835,6.279,7.1820,6.8880,6.279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39452,11/10/2022 21:30,1.646,21.000,22.050,21.000,23.1000,22.050,25.2000,24.1500,22.050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39453,11/10/2022 22:00,-0.188,20.160,21.168,20.160,22.1760,21.168,24.1920,23.1840,21.168,...,0.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
39454,11/10/2022 22:30,0.178,15.540,16.317,15.540,17.0940,16.317,18.6480,17.8710,16.317,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
column_names = df.columns.tolist()
print(column_names)

['UTC_Settlement_DateTime', 'battery_output', 'East_Midlands_price', 'Eastern_England_price', 'London_price', 'Merseyside_and_Northern_Wales_price', 'North_Western_England_price', 'Northern_Scotland_price', 'South_Western_England_price', 'Southern_England_price', 'Southern_Scotland_price', 'West_Midlands_price', 'Yorkshire_price', 'temperature_2mBelfast_weather', 'relativehumidity_2mBelfast_weather', 'dewpoint_2mBelfast_weather', 'apparent_temperatureBelfast_weather', 'precipitationBelfast_weather', 'rainBelfast_weather', 'snowfallBelfast_weather', 'weathercodeBelfast_weather', 'surface_pressureBelfast_weather', 'cloudcoverBelfast_weather', 'cloudcover_lowBelfast_weather', 'cloudcover_midBelfast_weather', 'cloudcover_highBelfast_weather', 'windspeed_10mBelfast_weather', 'windspeed_100mBelfast_weather', 'winddirection_10mBelfast_weather', 'winddirection_100mBelfast_weather', 'windgusts_10mBelfast_weather', 'is_dayBelfast_weather', 'locationBelfast_weather', 'populationBelfast_weather', 

In [5]:
print('Column_names: ')
for name in column_names:
    print(f'{name}')

Column_names: 
UTC_Settlement_DateTime
battery_output
East_Midlands_price
Eastern_England_price
London_price
Merseyside_and_Northern_Wales_price
North_Western_England_price
Northern_Scotland_price
South_Western_England_price
Southern_England_price
Southern_Scotland_price
West_Midlands_price
Yorkshire_price
temperature_2mBelfast_weather
relativehumidity_2mBelfast_weather
dewpoint_2mBelfast_weather
apparent_temperatureBelfast_weather
precipitationBelfast_weather
rainBelfast_weather
snowfallBelfast_weather
weathercodeBelfast_weather
surface_pressureBelfast_weather
cloudcoverBelfast_weather
cloudcover_lowBelfast_weather
cloudcover_midBelfast_weather
cloudcover_highBelfast_weather
windspeed_10mBelfast_weather
windspeed_100mBelfast_weather
winddirection_10mBelfast_weather
winddirection_100mBelfast_weather
windgusts_10mBelfast_weather
is_dayBelfast_weather
locationBelfast_weather
populationBelfast_weather
latitudeBelfast_weather
longitudeBelfast_weather
generationtime_msBelfast_weather
utc_of

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39457 entries, 0 to 39456
Columns: 293 entries, UTC_Settlement_DateTime to elevationSouthhampton - Portsmouth_weather
dtypes: float64(262), object(31)
memory usage: 88.5+ MB


In [7]:
df.select_dtypes(include=['object'])

,UTC_Settlement_DateTime,locationBelfast_weather,timezoneBelfast_weather,timezone_abbreviationBelfast_weather,locationBirmingham_weather,timezoneBirmingham_weather,timezone_abbreviationBirmingham_weather,locationGlasgow_weather,timezoneGlasgow_weather,timezone_abbreviationGlasgow_weather,...,timezone_abbreviationManchester_weather,locationNewcastle upon Tyne_weather,timezoneNewcastle upon Tyne_weather,timezone_abbreviationNewcastle upon Tyne_weather,locationSheffield_weather,timezoneSheffield_weather,timezone_abbreviationSheffield_weather,locationSouthhampton - Portsmouth_weather,timezoneSouthhampton - Portsmouth_weather,timezone_abbreviationSouthhampton - Portsmouth_weather
id,,,,,,,,,,,,,,,,,,,,,
0,8/10/2020 23:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8/11/2020 0:00,Belfast,GMT,GMT,Birmingham,GMT,GMT,Glasgow,GMT,GMT,...,GMT,Newcastle upon Tyne,GMT,GMT,Sheffield,GMT,GMT,Southhampton - Portsmouth,GMT,GMT
2,8/11/2020 0:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8/11/2020 1:00,Belfast,GMT,GMT,Birmingham,GMT,GMT,Glasgow,GMT,GMT,...,GMT,Newcastle upon Tyne,GMT,GMT,Sheffield,GMT,GMT,Southhampton - Portsmouth,GMT,GMT
4,8/11/2020 1:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39452,11/10/2022 21:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39453,11/10/2022 22:00,Belfast,GMT,GMT,Birmingham,GMT,GMT,Glasgow,GMT,GMT,...,GMT,Newcastle upon Tyne,GMT,GMT,Sheffield,GMT,GMT,Southhampton - Portsmouth,GMT,GMT
39454,11/10/2022 22:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


So we have some 31 columns that are object and need to be translated into numbers with some kind of encoding. Also we need to sort out the time series element of the dataframe

In [8]:
df.isna().any().head(20)

UTC_Settlement_DateTime                False
battery_output                         False
East_Midlands_price                    False
Eastern_England_price                  False
London_price                           False
Merseyside_and_Northern_Wales_price    False
North_Western_England_price            False
Northern_Scotland_price                False
South_Western_England_price            False
Southern_England_price                 False
Southern_Scotland_price                False
West_Midlands_price                     True
Yorkshire_price                        False
temperature_2mBelfast_weather           True
relativehumidity_2mBelfast_weather      True
dewpoint_2mBelfast_weather              True
apparent_temperatureBelfast_weather     True
precipitationBelfast_weather            True
rainBelfast_weather                     True
snowfallBelfast_weather                 True
dtype: bool

The west midlands price has NANs in it, this also needs to be sorted. We will do some inputation using the mean for this or any other method as appropiate

In [9]:
missing_values = 'West_Midlands_price'
mean_value = df['West_Midlands_price'].mean()
df['West_Midlands_price'] = df['West_Midlands_price'].fillna(mean_value)

In [10]:
df.isna().any().head(20)

UTC_Settlement_DateTime                False
battery_output                         False
East_Midlands_price                    False
Eastern_England_price                  False
London_price                           False
Merseyside_and_Northern_Wales_price    False
North_Western_England_price            False
Northern_Scotland_price                False
South_Western_England_price            False
Southern_England_price                 False
Southern_Scotland_price                False
West_Midlands_price                    False
Yorkshire_price                        False
temperature_2mBelfast_weather           True
relativehumidity_2mBelfast_weather      True
dewpoint_2mBelfast_weather              True
apparent_temperatureBelfast_weather     True
precipitationBelfast_weather            True
rainBelfast_weather                     True
snowfallBelfast_weather                 True
dtype: bool

Ok, we have sorted the problem of the missing values on the prize section, now onto the time series element

In [11]:
df['UTC_Settlement_DateTime'] = pd.to_datetime(df['UTC_Settlement_DateTime'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39457 entries, 0 to 39456
Columns: 293 entries, UTC_Settlement_DateTime to elevationSouthhampton - Portsmouth_weather
dtypes: datetime64[ns](1), float64(262), object(30)
memory usage: 88.5+ MB


In [12]:
df.set_index("UTC_Settlement_DateTime", inplace=True)
df.sort_index(inplace=True)


In [13]:
df

,battery_output,East_Midlands_price,Eastern_England_price,London_price,Merseyside_and_Northern_Wales_price,North_Western_England_price,Northern_Scotland_price,South_Western_England_price,Southern_England_price,Southern_Scotland_price,...,is_daySouthhampton - Portsmouth_weather,locationSouthhampton - Portsmouth_weather,populationSouthhampton - Portsmouth_weather,latitudeSouthhampton - Portsmouth_weather,longitudeSouthhampton - Portsmouth_weather,generationtime_msSouthhampton - Portsmouth_weather,utc_offset_secondsSouthhampton - Portsmouth_weather,timezoneSouthhampton - Portsmouth_weather,timezone_abbreviationSouthhampton - Portsmouth_weather,elevationSouthhampton - Portsmouth_weather
UTC_Settlement_DateTime,,,,,,,,,,,,,,,,,,,,,
2020-08-10 23:30:00,1.528,5.985,6.279,5.985,6.5835,6.279,7.1820,6.8880,6.279,6.279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-11 00:00:00,0.178,6.300,6.615,6.300,6.9300,6.615,7.5600,7.2450,6.615,6.615,...,0.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2020-08-11 00:30:00,-1.242,6.300,6.615,6.300,6.9300,6.615,7.5600,7.2450,6.615,6.615,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-11 01:00:00,2.104,5.985,6.279,5.985,6.5835,6.279,7.1820,6.8880,6.279,6.279,...,0.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2020-08-11 01:30:00,-0.664,5.985,6.279,5.985,6.5835,6.279,7.1820,6.8880,6.279,6.279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-10 21:30:00,1.646,21.000,22.050,21.000,23.1000,22.050,25.2000,24.1500,22.050,22.050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-10 22:00:00,-0.188,20.160,21.168,20.160,22.1760,21.168,24.1920,23.1840,21.168,21.168,...,0.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2022-11-10 22:30:00,0.178,15.540,16.317,15.540,17.0940,16.317,18.6480,17.8710,16.317,16.317,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df.rename(columns= {'battery_output': 'Target'}, inplace=True)
#df.sample(3)

In [15]:
df

,Target,East_Midlands_price,Eastern_England_price,London_price,Merseyside_and_Northern_Wales_price,North_Western_England_price,Northern_Scotland_price,South_Western_England_price,Southern_England_price,Southern_Scotland_price,...,is_daySouthhampton - Portsmouth_weather,locationSouthhampton - Portsmouth_weather,populationSouthhampton - Portsmouth_weather,latitudeSouthhampton - Portsmouth_weather,longitudeSouthhampton - Portsmouth_weather,generationtime_msSouthhampton - Portsmouth_weather,utc_offset_secondsSouthhampton - Portsmouth_weather,timezoneSouthhampton - Portsmouth_weather,timezone_abbreviationSouthhampton - Portsmouth_weather,elevationSouthhampton - Portsmouth_weather
UTC_Settlement_DateTime,,,,,,,,,,,,,,,,,,,,,
2020-08-10 23:30:00,1.528,5.985,6.279,5.985,6.5835,6.279,7.1820,6.8880,6.279,6.279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-11 00:00:00,0.178,6.300,6.615,6.300,6.9300,6.615,7.5600,7.2450,6.615,6.615,...,0.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2020-08-11 00:30:00,-1.242,6.300,6.615,6.300,6.9300,6.615,7.5600,7.2450,6.615,6.615,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-11 01:00:00,2.104,5.985,6.279,5.985,6.5835,6.279,7.1820,6.8880,6.279,6.279,...,0.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2020-08-11 01:30:00,-0.664,5.985,6.279,5.985,6.5835,6.279,7.1820,6.8880,6.279,6.279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-10 21:30:00,1.646,21.000,22.050,21.000,23.1000,22.050,25.2000,24.1500,22.050,22.050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-10 22:00:00,-0.188,20.160,21.168,20.160,22.1760,21.168,24.1920,23.1840,21.168,21.168,...,0.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2022-11-10 22:30:00,0.178,15.540,16.317,15.540,17.0940,16.317,18.6480,17.8710,16.317,16.317,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df = df[df.columns.drop(list(df.filter(regex='is_')))]


We have used this code to get rid of all the columns that start with is_ as it adds nothing in principle, we might do something along those lines later on

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 39457 entries, 2020-08-10 23:30:00 to 2022-11-10 23:30:00
Columns: 282 entries, Target to elevationSouthhampton - Portsmouth_weather
dtypes: float64(252), object(30)
memory usage: 85.2+ MB


In [18]:
df.head(50)

,Target,East_Midlands_price,Eastern_England_price,London_price,Merseyside_and_Northern_Wales_price,North_Western_England_price,Northern_Scotland_price,South_Western_England_price,Southern_England_price,Southern_Scotland_price,...,windgusts_10mSouthhampton - Portsmouth_weather,locationSouthhampton - Portsmouth_weather,populationSouthhampton - Portsmouth_weather,latitudeSouthhampton - Portsmouth_weather,longitudeSouthhampton - Portsmouth_weather,generationtime_msSouthhampton - Portsmouth_weather,utc_offset_secondsSouthhampton - Portsmouth_weather,timezoneSouthhampton - Portsmouth_weather,timezone_abbreviationSouthhampton - Portsmouth_weather,elevationSouthhampton - Portsmouth_weather
UTC_Settlement_DateTime,,,,,,,,,,,,,,,,,,,,,
2020-08-10 23:30:00,1.528,5.985,6.2790,5.985,6.5835,6.2790,7.1820,6.8880,6.2790,6.2790,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-11 00:00:00,0.178,6.300,6.6150,6.300,6.9300,6.6150,7.5600,7.2450,6.6150,6.6150,...,6.8,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2020-08-11 00:30:00,-1.242,6.300,6.6150,6.300,6.9300,6.6150,7.5600,7.2450,6.6150,6.6150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-11 01:00:00,2.104,5.985,6.2790,5.985,6.5835,6.2790,7.1820,6.8880,6.2790,6.2790,...,4.7,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2020-08-11 01:30:00,-0.664,5.985,6.2790,5.985,6.5835,6.2790,7.1820,6.8880,6.2790,6.2790,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-11 02:00:00,0.612,5.985,6.2790,5.985,6.5835,6.2790,7.1820,6.8880,6.2790,6.2790,...,4.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2020-08-11 02:30:00,2.206,5.985,6.2790,5.985,6.5835,6.2790,7.1820,6.8880,6.2790,6.2790,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-11 03:00:00,0.930,6.216,6.5310,6.216,6.8355,6.5310,7.4550,7.1505,6.5310,6.5310,...,4.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2020-08-11 03:30:00,0.146,6.090,6.3945,6.090,6.6990,6.3945,7.3080,7.0035,6.3945,6.3945,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
for col in df.columns:
    most_common_value = df[col].mode()[0]
    df[col].fillna(most_common_value, inplace=True)

C:\Users\ivan.sanz\AppData\Local\Temp\ipykernel_20824\955556545.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col].fillna(most_common_value, inplace=True)


In [20]:
df.tail(7)

,Target,East_Midlands_price,Eastern_England_price,London_price,Merseyside_and_Northern_Wales_price,North_Western_England_price,Northern_Scotland_price,South_Western_England_price,Southern_England_price,Southern_Scotland_price,...,windgusts_10mSouthhampton - Portsmouth_weather,locationSouthhampton - Portsmouth_weather,populationSouthhampton - Portsmouth_weather,latitudeSouthhampton - Portsmouth_weather,longitudeSouthhampton - Portsmouth_weather,generationtime_msSouthhampton - Portsmouth_weather,utc_offset_secondsSouthhampton - Portsmouth_weather,timezoneSouthhampton - Portsmouth_weather,timezone_abbreviationSouthhampton - Portsmouth_weather,elevationSouthhampton - Portsmouth_weather
UTC_Settlement_DateTime,,,,,,,,,,,,,,,,,,,,,
2022-11-10 20:30:00,1.036,21.714,22.7955,21.714,23.8875,22.7955,26.0610,24.9690,22.7955,22.7955,...,31.7,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2022-11-10 21:00:00,1.716,22.680,23.8140,22.680,24.9480,23.8140,27.2160,26.0820,23.8140,23.8140,...,43.2,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2022-11-10 21:30:00,1.646,21.000,22.0500,21.000,23.1000,22.0500,25.2000,24.1500,22.0500,22.0500,...,31.7,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2022-11-10 22:00:00,-0.188,20.160,21.1680,20.160,22.1760,21.1680,24.1920,23.1840,21.1680,21.1680,...,45.7,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2022-11-10 22:30:00,0.178,15.540,16.3170,15.540,17.0940,16.3170,18.6480,17.8710,16.3170,16.3170,...,31.7,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2022-11-10 23:00:00,-1.498,2.877,3.0240,2.877,3.1605,3.0240,3.4545,3.3075,3.0240,3.0240,...,46.8,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2022-11-10 23:30:00,-3.086,-2.877,-3.0240,-2.877,-3.1605,-3.0240,-3.4545,-3.3075,-3.0240,-3.0240,...,31.7,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0


Ok, this last line of code was not be best but it has worked so now lets do the one hot encoding so we have all numbers

In [21]:
df = df[df.columns.drop(list(df.filter(regex='timezone')))]

In [22]:
df

,Target,East_Midlands_price,Eastern_England_price,London_price,Merseyside_and_Northern_Wales_price,North_Western_England_price,Northern_Scotland_price,South_Western_England_price,Southern_England_price,Southern_Scotland_price,...,winddirection_10mSouthhampton - Portsmouth_weather,winddirection_100mSouthhampton - Portsmouth_weather,windgusts_10mSouthhampton - Portsmouth_weather,locationSouthhampton - Portsmouth_weather,populationSouthhampton - Portsmouth_weather,latitudeSouthhampton - Portsmouth_weather,longitudeSouthhampton - Portsmouth_weather,generationtime_msSouthhampton - Portsmouth_weather,utc_offset_secondsSouthhampton - Portsmouth_weather,elevationSouthhampton - Portsmouth_weather
UTC_Settlement_DateTime,,,,,,,,,,,,,,,,,,,,,
2020-08-10 23:30:00,1.528,5.985,6.279,5.985,6.5835,6.279,7.1820,6.8880,6.279,6.279,...,229.0,233.0,31.7,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,10.0
2020-08-11 00:00:00,0.178,6.300,6.615,6.300,6.9300,6.615,7.5600,7.2450,6.615,6.615,...,297.0,297.0,6.8,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,10.0
2020-08-11 00:30:00,-1.242,6.300,6.615,6.300,6.9300,6.615,7.5600,7.2450,6.615,6.615,...,229.0,233.0,31.7,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,10.0
2020-08-11 01:00:00,2.104,5.985,6.279,5.985,6.5835,6.279,7.1820,6.8880,6.279,6.279,...,333.0,11.0,4.7,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,10.0
2020-08-11 01:30:00,-0.664,5.985,6.279,5.985,6.5835,6.279,7.1820,6.8880,6.279,6.279,...,229.0,233.0,31.7,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-10 21:30:00,1.646,21.000,22.050,21.000,23.1000,22.050,25.2000,24.1500,22.050,22.050,...,229.0,233.0,31.7,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,10.0
2022-11-10 22:00:00,-0.188,20.160,21.168,20.160,22.1760,21.168,24.1920,23.1840,21.168,21.168,...,241.0,238.0,45.7,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,10.0
2022-11-10 22:30:00,0.178,15.540,16.317,15.540,17.0940,16.317,18.6480,17.8710,16.317,16.317,...,229.0,233.0,31.7,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,10.0


In [23]:
 

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Example list of strings
data = ['Belfast', 'Birmingham', 'Glasgow', 'Leeds', 'Liverpool', 'London', 'Manchester', 'Newcastle upon Tyne', 'Sheffield' , 'Southhampton - Portsmouth' ]
# Convert the list of strings into numerical values
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(data)
# Perform one-hot encoding
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
# Create dictionary with original values as keys and one-hot encoded values as values
hot_encoded_dict = {k:v for k,v in zip(data, onehot_encoded)}
hot_encoded_dict

c:\Users\ivan.sanz\AppData\Local\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


{'Belfast': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'Birmingham': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'Glasgow': array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 'Leeds': array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 'Liverpool': array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 'London': array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 'Manchester': array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]),
 'Newcastle upon Tyne': array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]),
 'Sheffield': array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]),
 'Southhampton - Portsmouth': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])}

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 39457 entries, 2020-08-10 23:30:00 to 2022-11-10 23:30:00
Columns: 262 entries, Target to elevationSouthhampton - Portsmouth_weather
dtypes: float64(252), object(10)
memory usage: 79.2+ MB


Ok after some consideration I am going to drop all the object columns as I don't think they are that relevant. The data already maps to the location so yeah.

In [25]:
df = df.select_dtypes(exclude=['object'])

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 39457 entries, 2020-08-10 23:30:00 to 2022-11-10 23:30:00
Columns: 252 entries, Target to elevationSouthhampton - Portsmouth_weather
dtypes: float64(252)
memory usage: 76.2 MB


In [27]:
df.to_csv('Training_sorted.csv', index=True)